# Azure AI Search: vector search, step by step

## Setup API client


In [1]:
import os
import dotenv
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv()

AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
AZURE_SEARCH_SERVICE_KEY = os.getenv("AZURE_SEARCH_SERVICE_KEY")

search_service_cred = AzureKeyCredential(AZURE_SEARCH_SERVICE_KEY)
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=search_service_cred)

## Tìm kiếm trên index

### Tạo index
- Bước đầu tiên chúng ta phải tạo một index.
- Index giống như collection chứa các tài liệu
- Chúng ta cần định nghĩa:
    - Cấu trúc của index: các field và kiểu dữ liệu
    - Thuật toán tìm kiếm trên vector
- Đoạn mã sau tạo ra index có 2 field:
    - id: id của document
    - embedding: là vector đại diện cho document, ở đây chúng ta định nghĩa số chiều của vector (vector_search_dimensions) là 3 

In [6]:
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    HnswParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
)

AZURE_SEARCH_TINY_INDEX = "teeenytinyindex"

index = SearchIndex(
    name=AZURE_SEARCH_TINY_INDEX, 
    fields=[
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchField(name="embedding", 
                    type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                    searchable=True, 
                    vector_search_dimensions=3,
                    vector_search_profile_name="embedding_profile")
    ],
    vector_search=VectorSearch(
        algorithms=[HnswAlgorithmConfiguration( # Hierachical Navigable Small World, IVF
                            name="hnsw_config",
                            kind=VectorSearchAlgorithmKind.HNSW,
                            parameters=HnswParameters(metric="cosine"),
                        )],
        profiles=[VectorSearchProfile(name="embedding_profile", algorithm_configuration_name="hnsw_config")]
    )
)

index_client.create_index(index)

### Thêm một vài document vào index

In [7]:
from azure.search.documents import SearchClient

search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_TINY_INDEX, credential=search_service_cred)
search_client.upload_documents(documents=[
    {"id": "1", "embedding": [1, 2, 3]},
    {"id": "2", "embedding": [1, 1, 3]},
    {"id": "3", "embedding": [4, 5, 6]}])

### Tìm kiếm với vector similarity

In [10]:
from azure.search.documents.models import VectorizedQuery

r = search_client.search(search_text=None, vector_queries=[
    VectorizedQuery(vector=[1, 2, 3], k_nearest_neighbors=3, fields="embedding")])
for doc in r:
    print(f"id: {doc['id']}, score: {doc['@search.score']}")

id: 1, score: 0.9999999
id: 3, score: 0.9752594
id: 2, score: 0.9680425


## Tìm kiếm với index lớn hơn
Giả sử chúng ta có thông tin sản phẩm như trong file products.txt, chúng ta cần lưu trữ các sản phẩm trong vector DB là Azure AI Search.

In [32]:
products = []
with open("../data/products.txt", "r") as f:
    for line in f:
        products.append(line)

In [33]:
products

['Lenovo Điện thoại, $405.00, Điện thoại thông minh với nhiều tính năng hiện đại và thiết kế đẹp mắt. Sản phẩm này có màn hình sắc nét, camera chất lượng cao và thời lượng pin lâu dài, phù hợp cho cả công việc và giải trí.\n',
 'Asus Máy chiếu, $1594.00, Máy chiếu với độ phân giải cao, phù hợp cho các buổi thuyết trình và giải trí. Sản phẩm này có khả năng trình chiếu rõ nét, màu sắc sống động và dễ dàng kết nối với các thiết bị khác.\n',
 'Asus Máy chơi game, $1951.00, Máy chơi game với đồ họa đẹp và nhiều trò chơi hấp dẫn. Sản phẩm này có khả năng chơi game mượt mà, đồ họa sắc nét và nhiều tính năng giải trí khác.\n',
 'Samsung Máy ảnh kỹ thuật số, $1376.00, Máy ảnh kỹ thuật số với khả năng chống nước, phù hợp cho các chuyến du lịch và hoạt động ngoài trời.\n',
 'Sony Máy chiếu, $1261.00, Máy chiếu với độ phân giải cao, phù hợp cho các buổi thuyết trình và giải trí. Sản phẩm này có khả năng trình chiếu rõ nét, màu sắc sống động và dễ dàng kết nối với các thiết bị khác.\n',
 'Dell Điệ

In [23]:
import dotenv
import openai

dotenv.load_dotenv()

# Initialize Azure search variables
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com")

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

Trước tiên, tạo một index khác.

Index bao gồm các field:
- id
- content: Lưu thông tin sản phẩm
- sourcefile: Lưu tên file để sử dụng cho việc trích dẫn sau này
- embedding: Lưu vector đại diện cho thông tin sản phẩm

Lưu ý: Kích thước vector là 1536 nếu sử dụng mô hình text-embedding-ada-002

In [34]:
from azure.search.documents.indexes.models import SearchableField
AZURE_SEARCH_FULL_INDEX = "gptkbindex"

index = SearchIndex(
    name=AZURE_SEARCH_FULL_INDEX, 
    fields=[
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type=SearchFieldDataType.String),
        SearchableField(name="sourcefile", type=SearchFieldDataType.String),
        SearchField(name="embedding", 
                    type=SearchFieldDataType.Collection(SearchFieldDataType.Single), 
                    searchable=True, 
                    vector_search_dimensions=1536,
                    vector_search_profile_name="embedding_profile")
    ],
    vector_search=VectorSearch(
        algorithms=[HnswAlgorithmConfiguration( # Hierachical Navigable Small World, IVF
                            name="hnsw_config",
                            kind=VectorSearchAlgorithmKind.HNSW,
                            parameters=HnswParameters(metric="cosine"),
                        )],
        profiles=[VectorSearchProfile(name="embedding_profile", algorithm_configuration_name="hnsw_config")]
    )
)

index_client.create_index(index)

Bước tiếp theo, chúng ta sẽ cần chuẩn bị danh sách documents từ products

In [35]:
documents = [{"id": str(i), "content": prod, "embedding": get_embedding(prod), "sourcefile": "products.txt"} for i, prod in enumerate(products)]

In [36]:
documents[0]

{'id': '0',
 'content': 'Lenovo Điện thoại, $405.00, Điện thoại thông minh với nhiều tính năng hiện đại và thiết kế đẹp mắt. Sản phẩm này có màn hình sắc nét, camera chất lượng cao và thời lượng pin lâu dài, phù hợp cho cả công việc và giải trí.\n',
 'embedding': [-0.009237738326191902,
  0.008772023022174835,
  0.025773799046874046,
  -0.029142256826162338,
  -0.04264160245656967,
  -0.006277578882873058,
  -0.011126115918159485,
  -0.007508852053433657,
  -0.02050420641899109,
  0.0023269786033779383,
  0.027381472289562225,
  0.04687769338488579,
  -0.019572775810956955,
  -0.013473828323185444,
  -0.02277536317706108,
  0.00010177541116718203,
  0.005352529231458902,
  -0.005422705318778753,
  0.022226711735129356,
  -0.024229923263192177,
  -0.029652629047632217,
  0.008427522145211697,
  0.0005374858155846596,
  0.008867718279361725,
  -0.009339812211692333,
  0.0009657200425863266,
  0.012229795567691326,
  -0.008976172655820847,
  -0.004153154324740171,
  -0.001827770727686584,

Upload document lên index

In [37]:

search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=search_service_cred)
search_client.upload_documents(documents=documents)

### Bây giờ chúng ta có thể tìm kiếm thông tin sản phẩm

In [40]:
search_query = "cho tôi xem máy in màu"
search_vector = get_embedding(search_query)
r = search_client.search(search_text=None, top=5, vector_queries=[
    VectorizedQuery(vector=search_vector, k_nearest_neighbors=5, fields="embedding")])
for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tContent: {content}")

Score: 0.88687	Content: Sony Máy in, $654.00, Máy in màu với độ phân giải cao, phù hợp cho in ấn hình ảnh và tài liệu màu. 
Score: 0.88352	Content: Canon Máy in, $901.00, Máy in màu với độ phân giải cao, phù hợp cho in ấn hình ảnh và tài liệu màu. 
Score: 0.88164	Content: Sony Máy in, $548.00, Máy in chất lượng cao, in ấn nhanh chóng và tiết kiệm mực. Sản phẩm này có khả năng in ấn đa dạng, từ tài liệu văn phòng đến hìn
Score: 0.88103	Content: Samsung Máy in, $401.00, Máy in chất lượng cao, in ấn nhanh chóng và tiết kiệm mực. Sản phẩm này có khả năng in ấn đa dạng, từ tài liệu văn phòng đến 
Score: 0.87831	Content: Nikon Máy in, $923.00, Máy in màu với độ phân giải cao, phù hợp cho in ấn hình ảnh và tài liệu màu. 
